In [ ]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio import SeqIO
from Bio.Seq import Seq

import numpy as np
import matplotlib.pyplot as plt
import os
import time

consensus = []
range_variation = []
nucleotide_count =[]
plocation = []
i_plocation = []
l_plocation = []

# Range for creating plotting colors
range1 = 50
range2 = 500
range3 = 2500

# Mutation threshold
m_t = 50

sizeb = 0

fip = 0
rfip = 0
a_end = 0

def ComputeMutation (p, nc_i ,i):
    if n > m_t:
        
        mutated_nt.seq[p] += " " + str(nc_i-1) + " "
        output_s.seq[p] = output_s.seq[p] + "\n" + str(nc_i) + "\t" + str(n) + "\t" + "\t".join(nucleotide_count[i].split('\t')[1:6])


        
class sequences:
    def __init__(self,p):
        self.seq = {
            "F3":p,
            "B3":p,
            "LF":p,
            "LB":p,
            "FIP":p,
            "rFIP":p,
            "BIP":p,
            "rBIP":p
        }

p_start = sequences(0)
primers = sequences("")
primers_fragments = sequences("")
output_s = sequences("")
mutated_nt = sequences("")


print("________________ALIGNING PRIMERS_________________")
with open("consensusSeq.txt", 'r') as cons:
    for record in SeqIO.parse(cons, "fasta"):
        consensus = str(record.seq).upper()
    with open("primers.txt", 'r') as primersIO:
            for primersIO in SeqIO.parse(primersIO, "fasta"):
                if "F3" in primersIO.id:
                    
                    primers.seq["F3"] = primersIO.seq
                    
                    alignments = pairwise2.align.localms(consensus, primersIO.seq,2,-4, -10,-1)
                    print("----------" + primersIO.id + "----------")
                    print(format_alignment(*alignments[0]))
                    p_start.seq["F3"] = alignments[0][3]
                    primers_fragments.seq["F3"] = format_alignment(*alignments[0]).split(" ")[1].split("\n")[0]

                elif "B3" in primersIO.id:
                    
                    primers.seq["B3"] = primersIO.seq
                    
                    alignments = pairwise2.align.localms(consensus, primersIO.seq.reverse_complement(),2,-4, -10,-1)
                    print("----------" + primersIO.id + "----------")
                    print(format_alignment(*alignments[0]))
                    p_start.seq["B3"] = alignments[0][3]
                    primers_fragments.seq["B3"] = format_alignment(*alignments[0]).split(" ")[1].split("\n")[0]
                    
                    a_end = alignments[0][4]
                    
                if "LF" in primersIO.id:
                    
                    primers.seq["LF"] = primersIO.seq
                    
                    alignments = pairwise2.align.localms(consensus, primersIO.seq.reverse_complement(),2,-4, -10,-1)
                    print("----------" + primersIO.id + "----------")
                    print(format_alignment(*alignments[0]))
                    p_start.seq["LF"] = alignments[0][3]
                    primers_fragments.seq["LF"] = format_alignment(*alignments[0]).split(" ")[1].split("\n")[0]

                elif "LB" in primersIO.id:
                    
                    primers.seq["LB"] = primersIO.seq
                    
                    alignments = pairwise2.align.localms(consensus, primersIO.seq,2,-4, -10,-1)
                    print("----------" + primersIO.id + "----------")
                    print(format_alignment(*alignments[0]))
                    p_start.seq["LB"] = alignments[0][3]
                    primers_fragments.seq["LB"] = format_alignment(*alignments[0]).split(" ")[1].split("\n")[0]

                elif "FIP" in primersIO.id:
                    
                    primers.seq["FIP"] = primersIO.seq
                    
                    alignments = pairwise2.align.localms(consensus, primersIO.seq,2,-4, -10,-1)
                    print("----------" + primersIO.id + "----------")
                    print(format_alignment(*alignments[0]))
                    p_start.seq["FIP"] = alignments[0][3]
                    buff = format_alignment(*alignments[0]).split(" ")
                    primers_fragments.seq["FIP"] = buff[1].split("\n")[0]
                    
                    alignments = pairwise2.align.localms(consensus, primersIO.seq.reverse_complement(),2,-4, -10,-1)
                    print("----------REVERSE COMPLEMENT OF " + primersIO.id + "----------")
                    print(format_alignment(*alignments[0]))
                    buff = format_alignment(*alignments[0]).split(" ")
                    p_start.seq["rFIP"] = alignments[0][3]
                    primers_fragments.seq["rFIP"] = buff[1].split("\n")[0]
                    

                elif "BIP" in primersIO.id:
                    
                    primers.seq["BIP"] = primersIO.seq
                    
                    alignments = pairwise2.align.localms(consensus, primersIO.seq,2,-4, -10,-1)
                    print("----------REVERSE COMPLEMENT OF " + primersIO.id + "----------")
                    print(format_alignment(*alignments[0]))
                    p_start.seq["rBIP"] = alignments[0][3]
                    primers_fragments.seq["rBIP"] = format_alignment(*alignments[0]).split(" ")[1].split("\n")[0]
                    sizeb=len(primersIO.seq)
                    
                    alignments = pairwise2.align.localms(consensus, primersIO.seq.reverse_complement(),2,-4, -10,-1)
                    print("----------" + primersIO.id + "----------")
                    print(format_alignment(*alignments[0]))
                    p_start.seq["BIP"] = alignments[0][3]
                    primers_fragments.seq["BIP"] = format_alignment(*alignments[0]).split(" ")[1].split("\n")[0]
            
print("PRIMERS RANGE:" + str(p_start.seq["F3"]) + " - " + str(a_end))

print("________________COMPUTING VARIATION MATRIX_________________")

if not os.path.isdir("LampPrimers"):
    os.mkdir("LampPrimers")
f_name = int(round(time.time() * 1000))
os.mkdir("LampPrimers/%s" % (f_name))

with open("LampPrimers/" + str(f_name) + "/output.txt", 'w') as output:
    with open("Variation.txt", 'r') as variation:
        with open("nc_count.txt", 'r') as ncount:

            nc_variation = variation.readline()
            ncount.readline()
#             The +1 in the following line is because pairwise2 alignment bases number start at 0 and the nc_count & variation.txt starts at 1
            while not str(p_start.seq["F3"]+1) in nc_variation:
                nc_variation = variation.readline()
                ncount.readline()

            range_variation.append(float(nc_variation.split("\t")[1].split("\n")[0]))
            nucleotide_count.append(ncount.readline())

            while not str(a_end) in nc_variation:
                
                
                nc_variation = variation.readline()
                range_variation.append(float(nc_variation.split("\t")[1].split("\n")[0]))
                
                nucleotide_count.append(ncount.readline())
                
#                 print(nucleotide_count[len(nucleotide_count)-1])
#                 print(range_variation[len(nucleotide_count)-1])

            buff = []
            i = 0

            for n,i in zip(range_variation, range(0,len(range_variation))):
                #calculate variation
                if n < range1:
                    buff.append(0)
                elif n < range2:
                    buff.append(0.33)
                elif n < range3:
                    buff.append(0.66)
                else:
                    buff.append(1)

                #Do this region have primers?
            #             F3 & B3
                if i < len(str(primers_fragments.seq["F3"])): #F3
                    plocation.append(0.1)
                    nucleotide_index = i+1
                    ComputeMutation("F3", nucleotide_index, i)


                elif i > (p_start.seq["B3"]-p_start.seq["F3"]) and i < ((p_start.seq["B3"]-p_start.seq["F3"]) + len(str(primers_fragments.seq["B3"]))):
                    plocation.append(0.9)
                    nucleotide_index = len(primers_fragments.seq["B3"])-(i-(p_start.seq["B3"]-p_start.seq["F3"]))
                    ComputeMutation("B3", nucleotide_index, i)

                else:
                    plocation.append(0)


            #                 FIP AND BIP
                if i > (p_start.seq["FIP"]-p_start.seq["F3"]) and i < ((p_start.seq["FIP"]-p_start.seq["F3"]) + len(str(primers_fragments.seq["FIP"]))): 
                    i_plocation.append(0.3)
                    nucleotide_index = len(primers.seq["FIP"]) - (len(primers_fragments.seq["FIP"])- ((i+1) - (p_start.seq["FIP"] - p_start.seq["F3"])))
                    ComputeMutation("FIP", nucleotide_index, i)

                elif i > (p_start.seq["rFIP"]-p_start.seq["F3"]) and i < ((p_start.seq["rFIP"]-p_start.seq["F3"]) + len(str(primers_fragments.seq["rFIP"]))): 
                    i_plocation.append(0.3)
                    nucleotide_index = (p_start.seq["rFIP"]-p_start.seq["F3"]) + len(primers_fragments.seq["rFIP"]) - i
                    if nucleotide_index > len(primers_fragments.seq["rBIP"]):
                        ComputeMutation("rFIP", nucleotide_index, i)

                elif i > (p_start.seq["rBIP"]-p_start.seq["F3"]) and i < ((p_start.seq["rBIP"]-p_start.seq["F3"]) + len(str(primers_fragments.seq["rBIP"]))): 
                    i_plocation.append(0.6)
                    nucleotide_index = (i+1)-(p_start.seq["rBIP"]-p_start.seq["F3"])
                    ComputeMutation("rBIP", nucleotide_index, i)

                elif i > (p_start.seq["BIP"]-p_start.seq["F3"]) and i < ((p_start.seq["BIP"]-p_start.seq["F3"]) + len(str(primers_fragments.seq["BIP"]))): 
                    i_plocation.append(0.6)
                    nucleotide_index = len(primers.seq["BIP"]) - (i - (p_start.seq["BIP"]-p_start.seq["F3"]))
                    if nucleotide_index > len(primers_fragments.seq["rBIP"]):
                        ComputeMutation("BIP", nucleotide_index, i)

                else:
                    i_plocation.append(0)



            #                 LOOP PRIMERS   
                if i > (p_start.seq["LF"]-p_start.seq["F3"]) and i < ((p_start.seq["LF"]-p_start.seq["F3"]) + len(str(primers_fragments.seq["LF"]))):
                    l_plocation.append(0.5)
                    nucleotide_index = len(primers_fragments.seq["LF"])-(i-(p_start.seq["LF"]-p_start.seq["F3"]))
                    ComputeMutation("LF", nucleotide_index, i)

                elif i > (p_start.seq["LB"]-p_start.seq["F3"]) and i < ((p_start.seq["LB"]-p_start.seq["F3"]) + len(str(primers_fragments.seq["LB"]))):
                    l_plocation.append(0.5)
                    nucleotide_index = 1+i-(p_start.seq["LB"]-p_start.seq["F3"])
                    ComputeMutation("LB", nucleotide_index, i)
                else:
                    l_plocation.append(0)


            range_variation = buff

            rv = np.array([range_variation])
            rv = np.vstack((rv, np.array([plocation])))
            rv = np.vstack((rv, np.array([i_plocation])))
            rv = np.vstack((rv, np.array([l_plocation])))

            fig, ax = plt.subplots(figsize=( (a_end - p_start.seq["F3"])/10, 1))
            im = ax.imshow(rv,aspect='auto')


            plt.show()
            fig.savefig("LampPrimers/" + str(f_name) + "/heatmap.png")
            
            for n in primers.seq:
                if primers.seq[n] != "":
                    output.write(">" + n +"\n")
                    output.write(str(primers.seq[n]) + "\n")
            
            output.write("\n------------------------------------------------")
            output.write("\n|            MUTATION ANALYSIS                 |")
            output.write("\n------------------------------------------------\n")
            
#             Calculate FIP and BIP complete sequences
            
            output_s.seq["FIP"] = output_s.seq["rFIP"] + output_s.seq["FIP"]
            output_s.seq["BIP"] = output_s.seq["BIP"] + output_s.seq["rBIP"]
            
            mutated_nt.seq["FIP"] = mutated_nt.seq["rFIP"] + mutated_nt.seq["FIP"]
            mutated_nt.seq["BIP"] = mutated_nt.seq["BIP"] + mutated_nt.seq["rBIP"]
        
            for n in output_s.seq:
                        
                if output_s.seq[n] != "" and not "r" in n:
                    
                    output.write("\n" + str(n) + "\n")
                    output.write("------------------------------------------\n")
                    for nc,ni in zip(primers.seq[n],range(0,len(primers.seq[n]))):
                        if " " + str(ni) + " " in mutated_nt.seq[n]:
                            output.write(" *" + nc + "* ")
                        else:
                            output.write(nc)
                    output.write("\nMutated nucleotides count in the TARGET STRAND:")
                    output.write("\nnc\tvar.\tA\tT\tG\tC\t-\n")
                    output.write(output_s.seq[n])
                
            output.close()
            print("Row legend:\n\t- 1st: Mutation rate.\n\t- 2nd: B3 and F3 primers alignment.\n\t- 3rd: FIP and BIP primers alignment.\n\t- 4th: Loop primers alignment.")